In [1]:
import os
import json
import pandas as pd
os.chdir("/home/t-iswatts/Desktop/MultilingualBlanketEval")
os.getcwd()

'/home/t-iswatts/Desktop/MultilingualBlanketEval'

In [45]:
final_output = []

datasets = os.listdir("results")
ignore = ["IN22-Gen", "xrisawoz"]
missing = []
models = set()
for dataset in datasets:
    if dataset in ignore:
            continue
    for model in os.listdir(os.path.join("results/", dataset)):
        models.add(model)

print(models)

for model in models:
    dataset_name = []
    languages = []
    results = []
    metrics = []
    # strategies = []
    for dataset in datasets:
        if dataset in ignore:
            continue
        
        try:
            for language in os.listdir(os.path.join("results/", dataset, model)):
                if dataset == "XLSum":
                    if not language.endswith("predictions.csv"):
                        continue
                    try:
                        temp_df = pd.read_csv(os.path.join("results/", dataset, model, language))
                        dataset_name.append(dataset)
                        languages.append(language[:-16])
                        metrics.append("ROUGE-L")
                        # strategies.append("N/A")
                        score = temp_df["ROUGE-L"].mean()
                        if score > 1:
                            results.append(round(score/100, 3))
                        else:
                            results.append(round(score, 3))
                    except:
                        print(f"Model: {model} not Evaluated on Dataset: {dataset} with Language: {language}")
                        missing.append((model, dataset, language))
                else:
                    strategies = os.listdir(os.path.join("results/", dataset, model, language))
                    if len(strategies) > 1:
                        strategies = [val for val in strategies if "8" in val]
                    # if dataset == "indicqa":
                    #     print(strategies)
                    for strategy in strategies:
                        try:
                            with open(os.path.join("results/", dataset, model, language, strategy, "results.json"), "r") as f:
                                result = json.load(f)
                            
                            for key in list(result["metrics"].keys())[:2]:
                                dataset_name.append(dataset)
                                languages.append(language)
                                metrics.append(key)
                                if result["metrics"][key] > 1:
                                    results.append(round(result["metrics"][key]/100, 3))
                                else:
                                    results.append(round(result["metrics"][key], 3))
                                # strategies.append(strategy)
                        except:
                            print(f"Model: {model} not Evaluated on Dataset: {dataset} with Language: {language} and Strategy: {strategy}")
                            missing.append((model, dataset, language, strategy))
                    
        except Exception as e:
            print(e)
            # print(f"Model: {model} not Evaluated on Dataset: {dataset}")
            pass
        
    final_output.append(pd.DataFrame({"Dataset": dataset_name, "Language": languages, "Metric": metrics, model: results}))
        # except:
        #     print(f"Model: {model} not Evaluated on Dataset: {dataset}")

{'gemini-pro', 'gemma-7b-it', 'gemma-2b-it'}
[Errno 2] No such file or directory: 'results/indicqa/gemini-pro'
Model: gemini-pro not Evaluated on Dataset: udpos with Language: ja and Strategy: PivotLang_ja_PromptName_structure_prompting_chat_Verbalizer_identity_FewShotK_8wthInstruction
[Errno 2] No such file or directory: 'results/afriqa/gemini-pro'
[Errno 2] No such file or directory: 'results/panx/gemma-7b-it'
[Errno 2] No such file or directory: 'results/gluecos_sentiment/gemma-7b-it'


[Errno 2] No such file or directory: 'results/gluecos_nli/gemma-7b-it'
[Errno 2] No such file or directory: 'results/udpos/gemma-7b-it'
Model: gemma-7b-it not Evaluated on Dataset: xcopa with Language: en and Strategy: PivotLang_en_PromptName_plausible_alternatives_discrete_FewShotK_8
Model: gemma-7b-it not Evaluated on Dataset: xstory_cloze with Language: ar and Strategy: PivotLang_ar_PromptName_Answer Given options_Verbalizer_identity_FewShotK_8wthInstruction
Model: gemma-7b-it not Evaluated on Dataset: afriqa with Language: fon and Strategy: PivotLang_fon_PromptName_answer_given_context_and_question_Verbalizer_identity_FewShotK_8
[Errno 2] No such file or directory: 'results/panx/gemma-2b-it'
[Errno 2] No such file or directory: 'results/gluecos_sentiment/gemma-2b-it'
[Errno 2] No such file or directory: 'results/gluecos_nli/gemma-2b-it'
[Errno 2] No such file or directory: 'results/udpos/gemma-2b-it'
Model: gemma-2b-it not Evaluated on Dataset: pawsx with Language: ko and Strategy:

In [46]:
len(final_output)

3

In [47]:
final_output[0]

,Dataset,Language,Metric,gemini-pro
0,xquad,es,exact_match,0.596
1,xquad,es,f1,0.771
2,xquad,zh,exact_match,0.559
3,xquad,zh,f1,0.594
4,xquad,el,exact_match,0.524
...,...,...,...,...
182,tydiqa,en,f1,0.712
183,tydiqa,bn,exact_match,0.664
184,tydiqa,bn,f1,0.751
185,tydiqa,sw,exact_match,0.731


In [48]:
final_dataframe = final_output[0]
for i in range(1, len(final_output)):
    final_dataframe = final_dataframe.merge(final_output[i], on=["Dataset", "Language", "Metric"], how="outer")

In [ ]:
final_dataframe.to_csv("results.csv", index=False)

In [43]:
final_dataframe.to_excel("results.xlsx", index=False)

### Scale Sheet

In [2]:
megav3 = pd.read_excel("MEGA_list_of_languages 3.xlsx")
megav3.head()

,Dataset,Task,Language,Language Tier,Language Code,Language Class,Language Script,DV003 Monolingual,DV003 TranslateTest,DV003 Zero-Shot Cross Lingual,...,Mistral-7B-Instruct-v0.1,GPT4-V Monolingual,GPT4-V Translate-Test,Mistral-7B-v0.1,gemma-2b-it,gemma-7b-it,gemini-pro,Metrics_x,Direction,Metrics
0,AfriQA,Question Answering,Bemba,NaN,bem,0.0,Latin,NaN,NaN,NaN,...,1.9,NaN,NaN,1.7,NaN,NaN,NaN,F1 Score,HigherBetter,F1 Score
1,AfriQA,Question Answering,Fon,NaN,fon,0.0,Latin,NaN,NaN,NaN,...,5.6,NaN,NaN,0.0,0,0,NaN,F1 Score,HigherBetter,F1 Score
2,AfriQA,Question Answering,Hausa,NaN,ha,2.0,Latin,NaN,NaN,NaN,...,0.4,NaN,NaN,1.7,0,38.6,NaN,F1 Score,HigherBetter,F1 Score
3,AfriQA,Question Answering,Igbo,NaN,ig,1.0,Latin,NaN,NaN,NaN,...,0.1,NaN,NaN,1.4,0,0,NaN,F1 Score,HigherBetter,F1 Score
4,AfriQA,Question Answering,Kinyarwanda,NaN,rw,0.0,Latin,NaN,NaN,NaN,...,1.9,NaN,NaN,1.9,NaN,NaN,NaN,F1 Score,HigherBetter,F1 Score


In [3]:
models = megav3.columns[7:-3]
models

Index(['DV003 Monolingual', 'DV003 TranslateTest',
       'DV003 Zero-Shot Cross Lingual', 'IndicTrans2',
       'GPT-3.5-Turbo Monolingual', 'GPT-3.5-Turbo Translate-Test',
       'GPT-3.5-Turbo Zero-Shot Cross Lingual', 'GPT-4 Monolingual',
       'GPT-4 Translate-Test', 'GPT-4 Zero-Shot Cross Lingual', 'TULRv6',
       'BLOOMZ', 'XLM-R', 'mBERT', 'MuRIL', 'XGLM', 'mT5', 'PaLM2 Zero-Shot',
       'PaLM2 Monolingual', 'Llama2-7B Monolingual', 'Llama2-13B Monolingual',
       'Llama2-70B Monolingual', 'LLaVa-1.5-13B Monolingual',
       'LLaVa-1.5-13B Translate-Test', 'Bing Translator', 'Google Translator',
       'Systran', 'Mistral-7B-Instruct-v0.1', 'GPT4-V Monolingual',
       'GPT4-V Translate-Test', 'Mistral-7B-v0.1', 'gemma-2b-it',
       'gemma-7b-it', 'gemini-pro'],
      dtype='object')

In [4]:
google_models = models[-3:]
google_models

Index(['gemma-2b-it', 'gemma-7b-it', 'gemini-pro'], dtype='object')

In [30]:
datasets = megav3["Dataset"].unique()
print(datasets)
qa_datasets = ["AfriQA", "IndicQA", "MLQA", 'TyDiQA-GoldP', "XQuAD"]
qa_datasets

['AfriQA' 'Belebele' 'GLUECoS-NLI' 'GLUECoS-SA' 'IN22_conv_en_indic'
 'IN22_conv_indic_en' 'IN22_gen_indic_en' 'IN22_gen_en_indic' 'IndicQA'
 'IndicXNLI' 'Jigsaw' 'MARVL' 'MLQA' 'PAN-X' 'PAWS-X' 'TyDiQA-GoldP'
 'UDPOS' 'WinoMT' 'XCOPA' 'XLSum' 'XM3600' 'XNLI' 'XQuAD' 'X-RiSAWOZ'
 'XstoryCloze']


['AfriQA', 'IndicQA', 'MLQA', 'TyDiQA-GoldP', 'XQuAD']

In [69]:
qa_df = megav3[megav3["Dataset"] == qa_datasets[1]]
qa_df = qa_df[["Language"]+list(google_models)]
qa_df.head(10)

,Language,gemma-2b-it,gemma-7b-it,gemini-pro
91,Assamese,6.6/ 5.5,19.1/12.6,NaN
92,Bengali,4.5/2.9,21.9/14.9,NaN
93,Gujarati,10.1/9.6,22.2/20.1,NaN
94,Hindi,5.9/1.8,32/21.8,NaN
95,Kannada,12.6/11.6,21.8/17.4,NaN
96,Malayalam,4/3,15.6/12.6,NaN
97,Marathi,5.5/3.9,22.5/16.4,NaN
98,Odia,2.3/2.3,7.5/7.4,NaN
99,Punjabi,7.7/5.1,19.4/15,NaN
100,Tamil,5.6/2.4,24.3/18.1,NaN


In [70]:
eval_cols = []
for col in qa_df.columns[1:-1]:
    scores = [val.split("/") for val in qa_df[col]]
    qa_df[f"{col}_f1"] = [float(val[0]) for val in scores]
    qa_df[f"{col}_em"] = [float(val[1]) for val in scores]
    eval_cols.append(f"{col}_f1")
    eval_cols.append(f"{col}_em")

In [71]:
eval_cols

['gemma-2b-it_f1', 'gemma-2b-it_em', 'gemma-7b-it_f1', 'gemma-7b-it_em']

In [72]:
qa_df.head()

,Language,gemma-2b-it,gemma-7b-it,gemini-pro,gemma-2b-it_f1,gemma-2b-it_em,gemma-7b-it_f1,gemma-7b-it_em
91,Assamese,6.6/ 5.5,19.1/12.6,NaN,6.6,5.5,19.1,12.6
92,Bengali,4.5/2.9,21.9/14.9,NaN,4.5,2.9,21.9,14.9
93,Gujarati,10.1/9.6,22.2/20.1,NaN,10.1,9.6,22.2,20.1
94,Hindi,5.9/1.8,32/21.8,NaN,5.9,1.8,32.0,21.8
95,Kannada,12.6/11.6,21.8/17.4,NaN,12.6,11.6,21.8,17.4


In [73]:
result = qa_df[eval_cols].mean(axis=0).to_dict()
for key in result:
    result[key] = round(result[key], 1)


result

{'gemma-2b-it_f1': 6.2,
 'gemma-2b-it_em': 4.6,
 'gemma-7b-it_f1': 20.1,
 'gemma-7b-it_em': 15.4}

In [18]:
qa_df[list(google_models)].mean(axis=0)

gemma-2b-it    7.78
gemma-7b-it    16.7
gemini-pro      NaN
dtype: object

In [ ]:
for qa_dataset in qa_datasets:
    qa_df = megav3[megav3["Dataset"] == qa_dataset]
    qa_df = qa_df[list(google_models)]

    for col in qa_df.columns:
        scores = [val.split("/") for val in qa_df[col]]
        


In [23]:
for model in models:
    megav3[model] = megav3[model].apply(lambda x: round(x*100,1))

In [24]:
megav3.head()

,Dataset,Task,Language,Language Tier,Language Code,Language Class,Language Script,DV003 Monolingual,DV003 TranslateTest,DV003 Zero-Shot Cross Lingual,...,Mistral-7B-Instruct-v0.1,GPT4-V Monolingual,GPT4-V Translate-Test,Mistral-7B-v0.1,gemma-2b-it,gemma-7b-it,gemini-pro,Metrics_x,Direction,Metrics
0,AfriQA,Question Answering,Bemba,NaN,bem,0.0,Latin,NaN,NaN,NaN,...,1.9,NaN,NaN,1.7,NaN,NaN,NaN,F1 Score,HigherBetter,F1 Score
1,AfriQA,Question Answering,Fon,NaN,fon,0.0,Latin,NaN,NaN,NaN,...,5.6,NaN,NaN,0.0,0.0,0.0,NaN,F1 Score,HigherBetter,F1 Score
2,AfriQA,Question Answering,Hausa,NaN,ha,2.0,Latin,NaN,NaN,NaN,...,0.4,NaN,NaN,1.7,0.0,38.6,NaN,F1 Score,HigherBetter,F1 Score
3,AfriQA,Question Answering,Igbo,NaN,ig,1.0,Latin,NaN,NaN,NaN,...,0.1,NaN,NaN,1.4,0.0,0.0,NaN,F1 Score,HigherBetter,F1 Score
4,AfriQA,Question Answering,Kinyarwanda,NaN,rw,0.0,Latin,NaN,NaN,NaN,...,1.9,NaN,NaN,1.9,NaN,NaN,NaN,F1 Score,HigherBetter,F1 Score


In [25]:
megav3.to_excel("MEGA_list_of_languages 3_1.xlsx", index=False)